# Deglinting a Sentinel-2 granule

This notebook demonstrates the different deglinting approaches using the Hedley method:

* Hedley et al., 2005 (correlation approach)
* Hedley et al., 2019 (using specific bands for different resolution groups)

In [1]:
from pathlib import Path
import tempfile
import numpy
import matplotlib.pyplot as plt
import rasterio
import datacube
from sungc import deglint, visualise
from sungc import rasterio_funcs as rf

In [2]:
dc = datacube.Datacube(env="water-atcor", config="/g/data/r78/dea-atmospheric-correction-over-water/datacube.conf")

In [3]:
query = {
    "product": "ga_s2am_aard_2",
    "region_code": "55HBT",
    "time": ("2017-04-17", "2017-04-19")
}

The `sungc` module enables a user to supply an opendatacube `Dataset` object as returned by `find_datasets`.  The premise is that deglinting is more robust prior to resampling as opposed to resampling upfront that generally occur when using `dc.load`.

In [4]:
dataset = dc.find_datasets(**query)[0]

The `ga_s2am_aard_2` product is comprised of the following sub-products:

* lmbskyg `lambertian sky glint corrected`
* oa `observation attributes`

In the same way that the `ga_ls8c_ard_3` product which contains the following sub-products:

* nbar `nadir corrected brdf adjusted reflectance
* nbart `nadir corrected brdf adjusted reflectance terrain illumination correction
* oa `observation attributes`

In [5]:
sub_product = "lmbskyg"
g = deglint.GlintCorr(dataset, sub_product)

We'll save the generated Region Of Interest (ROI) for use later on

In [6]:
tempdir = tempfile.TemporaryDirectory(".tmp", "ROI-")
roi_out_fname = Path(tempdir.name).joinpath("deep-water-ROI.shp")

## Hedley 2005
The Hedley method requires a Region Of Interest to be selected (or provided) that contains both deep water and sun glint affected pixels.
This version is using Band-8 to deglint bands 1-5.

In [7]:
%matplotlib
hedley_xarr = g.hedley_2005(
    vis_bands=["1", "2", "3", "4", "5"],
    corr_band="8",
    roi_shpfile=roi_out_fname,
    water_val=5
)

Using matplotlib backend: Qt5Agg


A list of 3 `xarray.Dataset` objects for each of the resolution groups of Sentinel-2

* 60 metres
* 10 metres
* 20 metres

In [8]:
hedley_xarr

[<xarray.Dataset>
 Dimensions:                  (time: 1, x: 1830, y: 1830)
 Coordinates:
   * time                     (time) datetime64[ns] 2017-04-18T00:12:56.756000
     lat                      (y, x) float64 5.8e+06 5.8e+06 ... 5.69e+06
     lon                      (y, x) float64 2e+05 2.001e+05 ... 3.098e+05
 Dimensions without coordinates: x, y
 Data variables:
     lmbskyg_coastal_aerosol  (time, y, x) int16 -999 -999 -999 ... 298 289 277
 Attributes:
     deglint_algorithm:  Hedley et al. (2005)
     spatial_res:        60.0
     transform:          | 60.00, 0.00, 199980.00|\n| 0.00,-60.00, 5800000.00|...
     crs:                EPSG:32755,
 <xarray.Dataset>
 Dimensions:        (time: 1, x: 10980, y: 10980)
 Coordinates:
   * time           (time) datetime64[ns] 2017-04-18T00:12:56.756000
     lat            (y, x) float64 5.8e+06 5.8e+06 5.8e+06 ... 5.69e+06 5.69e+06
     lon            (y, x) float64 2e+05 2e+05 2e+05 ... 3.098e+05 3.098e+05
 Dimensions without coordinate

## Hedley et al. 2019
An alternate method for deglinting Sentinel-2 was proposed using the following bands:

* Band 9 to deglint Band 1 (both at 60 m)
* Band 7 to deglint Band 5 (20 m resolution)
* Band 8 to deglint Bands 2, 3 and 4 (10 m resolution)

As Band-9 isn't produced as part of the `ga_s2am_aard_2` and `ga_s2bm_aard_2` products due to the significant influence that water vapour has on the radiative transfer for Band-9, Band-8 can be used instead.

In [9]:
xarr_60m = g.hedley_2005(
    vis_bands=["1"],
    corr_band="8a",
    roi_shpfile=roi_out_fname,
    water_val=5
)[0]

In [10]:
xarr_20m = g.hedley_2005(
    vis_bands=["5"],
    corr_band="7",
    roi_shpfile=roi_out_fname,
    water_val=5
)[0]

In [11]:
xarr_10m = g.hedley_2005(
    vis_bands=["2", "3", "4"],
    corr_band="8",
    roi_shpfile=roi_out_fname,
    water_val=5
)[0]